In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import keras.models as models
import keras.utils as image
import tensorflow as tf
from tensorflow.keras.models import Model
from sklearn.metrics.pairwise import euclidean_distances

hognet_model = models.load_model("drive/MyDrive/Thesis/hog_predictor.hdf5")
#hognet_model.summary()
vpr_model = Model(inputs=hognet_model.input, outputs=hognet_model.get_layer('flatten').output) ## It is actually the features of the second pooling layer
vpr_model.summary()

def numerical_sort(value):
    parts = value.split('.')
    return int(parts[0])

def extract_features(input_image):
    img = cv2.imread(input_image, cv2.IMREAD_COLOR)
    resized_img = tf.image.resize(img, (256, 256))
    input_set = [resized_img]
    input_set = tf.convert_to_tensor(input_set, dtype=tf.int16)
    return vpr_model.predict(input_set)

reference_folder = 'drive/MyDrive/Thesis/Nordland/Summer'
files = os.listdir(reference_folder)
sorted_files = sorted(files, key=numerical_sort)
files = sorted_files[:100]

# Reference dataset (database) images
reference_images = [os.path.join(reference_folder, file) for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

'''
# Extract features from reference images
reference_features = [extract_features(image_path) for image_path in reference_images]

np.save('drive/MyDrive/Thesis/Nordland/reference_features_8000.npy', reference_features)
'''

reference_features = np.load('drive/MyDrive/Thesis/Nordland/reference_features_8000.npy')

print (reference_features.shape)

# Query image
query_folder = 'drive/MyDrive/Thesis/Nordland/Summer'

files = os.listdir(query_folder)
sorted_files = sorted(files, key=numerical_sort)
files = sorted_files[:100]

correct_predictions = 0
correct_image_pair_predictions = []
incorrect_image_pair_predictions = []
# Extract features from query image
for query_image in files:
    file = query_image
    query_image = os.path.join(query_folder, query_image)
    query_features = extract_features(query_image)

    # Calculate Euclidean distances between query features and reference features
    query_index = int(file.rstrip(".png"))
    distances = [euclidean_distances(query_features, ref_features)[0][0] for ref_features in reference_features]

    # Find the index of the closest image (smallest distance)
    closest_index = np.argmin(distances)
    closest_image_path = reference_images[closest_index]
    print(query_index)
    print(closest_index)
    #if abs(query_index - start - closest_index) <= 5:
    if abs(query_index - closest_index - 2000) <= 3:
        correct_predictions += 1
        correct_image_pair_predictions.append([query_image , closest_image_path])
    else:
        incorrect_image_pair_predictions.append([query_image , closest_image_path])

print (correct_predictions)

# Display query and closest image
query_img = plt.imread(correct_image_pair_predictions[10][0])
closest_img = plt.imread(correct_image_pair_predictions[10][1])

plt.subplot(1, 2, 1)
plt.imshow(query_img)
plt.title('Query Image')

plt.subplot(1, 2, 2)
plt.imshow(closest_img)
plt.title('Closest Image from Reference')

plt.show()

# Display another query and closest image
query_img = plt.imread(correct_image_pair_predictions[50][0])
closest_img = plt.imread(correct_image_pair_predictions[50][1])

plt.subplot(1, 2, 1)
plt.imshow(query_img)
plt.title('Query Image')

plt.subplot(1, 2, 2)
plt.imshow(closest_img)
plt.title('Closest Image from Reference')

plt.show()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_input (InputLayer)   [(None, 256, 256, 3)]     0         
                                                                 
 lambda (Lambda)             (None, 256, 256, 3)       0         
                                                                 
 lambda_1 (Lambda)           (None, 256, 256, 1)       0         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      9248      
                                                             

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
import numpy as np
import tensorflow as tf
from sklearn.metrics.pairwise import euclidean_distances
#from sklearn.metrics.pairwise import cosine_distances
import matplotlib.pyplot as plt
import os

# Load pre-trained VGG16 model (excluding the top fully-connected layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(480, 480, 3))

# Add Global Average Pooling layer
x = GlobalAveragePooling2D()(base_model.output)

# Create the VPR model
vpr_model = Model(inputs=base_model.input, outputs=x)

# Function to extract features from an image
def extract_features(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(480, 480))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.vgg16.preprocess_input(img_array)
    features = vpr_model.predict(img_array)
    return features

# Path to the folder containing reference images
reference_folder = 'drive/MyDrive/Thesis/full_merged_cam3-20240214T083003Z-001'
files = os.listdir(reference_folder)

# Reference dataset (database) images
reference_images = [os.path.join(reference_folder, file) for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Query image
#query_image = '1692872703659696197.png'
query_image = reference_images[203]

# Extract features from reference images
reference_features = [extract_features(image_path) for image_path in reference_images[:202]]

# Extract features from query image
query_features = extract_features(query_image)

# Calculate Euclidean distances between query features and reference features
distances = [euclidean_distances(query_features, ref_features)[0][0] for ref_features in reference_features]

# Find the index of the closest image (smallest distance)
closest_index = np.argmin(distances)
print(closest_index)
closest_image_path = reference_images[closest_index]

# Display query and closest image
query_img = plt.imread(query_image)
closest_img = plt.imread(closest_image_path)

plt.subplot(1, 2, 1)
plt.imshow(query_img)
plt.title('Query Image')

plt.subplot(1, 2, 2)
plt.imshow(closest_img)
plt.title('Closest Image from Reference')

plt.show()

1/1 [==============================] - 4s 4s/step


In [1]:
!sudo apt update

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [23.8 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,307 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,369 kB]
Get:13 

In [2]:
!sudo apt-get --purge -y remove 'nvidia*'

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'nvidia-driver-550-server' for glob 'nvidia*'
Note, selecting 'nvidia-firmware-535-535.154.05' for glob 'nvidia*'
Note, selecting 'nvidia-docker2' for glob 'nvidia*'
Note, selecting 'nvidia-cuda-toolkit-doc' for glob 'nvidia*'
Note, selecting 'nvidia-imex' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-firmware-535-server-535.154.05' for glob 'nvidia*'
Note, selecting 'nvidia-headless-390' for glob 'nvidia*'
Note, selecting 'nvidia-cuda-toolkit-gcc' for glob 'nvidia*'
Note, selecting 'nvidia-headless-418' for glob 'nvidia*'
Note, selecting 'nvidia-headless-430' for glob 'nvidia*'
Note, selecting 'nvidia-headless-435' for glob 'nvidia*'
Note, selecting 'nvidia-headless-440' for glob 'nvidia*'
Note, selecting 'nvidia-headless-450' for glob 'nvidia*'
Note, selecting 'nvidia-headless-455' for glob 'nvidia*'
Note, selecting 

In [3]:
!sudo apt install --reinstall nvidia-driver-535 nvidia-dkms-535

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages were automatically installed and are no longer required:
  ocl-icd-opencl-dev opencl-c-headers opencl-clhpp-headers
Use 'sudo apt autoremove' to remove them.
The following additional packages will be installed:
  cpp-12 dctrl-tools dkms fakeroot gcc-12 keyboard-configuration libasan8
  libfakeroot libfontenc1 libgcc-12-dev libjansson4 liblocale-gettext-perl
  libnvidia-cfg1-535 libnvidia-common-535 libnvidia-compute-535
  libnvidia-decode-535 libnvidia-encode-535 libnvidia-extra-535
  libnvidia-fbc1-535 libnvidia-gl-535 libtsan2 libudev1 libxcvt0 libxfont2
  libxkbfile1 nvidia-compute-utils-535 nvidia-kernel-common-535
  nvidia-kernel-source-535 nvidia-prime nvidia-settings nvidia-utils-535
  python3-xkit screen-resolution-extra systemd-hwe-hwdb udev x11-xkb-utils
  xcvt xfonts-base xfonts-encodings xfonts-utils xserver-common
  xserver-xorg-core xserver-xorg-video-nv

In [2]:
!apt-get install cuda-drivers

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages were automatically installed and are no longer required:
  ocl-icd-opencl-dev opencl-c-headers opencl-clhpp-headers
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  cuda-drivers-550 libnvidia-cfg1-550 libnvidia-common-550 libnvidia-compute-550
  libnvidia-decode-550 libnvidia-encode-550 libnvidia-extra-550 libnvidia-fbc1-550 libnvidia-gl-550
  nvidia-compute-utils-550 nvidia-dkms-550 nvidia-driver-550 nvidia-firmware-550-550.54.15
  nvidia-kernel-common-550 nvidia-kernel-source-550 nvidia-utils-550 xserver-xorg-video-nvidia-550
Recommended packages:
  libnvidia-compute-550:i386 libnvidia-decode-550:i386 libnvidia-encode-550:i386
  libnvidia-fbc1-550:i386 libnvidia-gl-550:i386
The following packages will be REMOVED:
  libnvidia-cfg1-535 libnvidia-common-535 libnvidia-compute-535 libnvidia-decode-535
  libnvidia-encode-535 libn

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found
